In [1]:
from twitterscraper import query_tweets
from datetime import datetime
from pandas import DataFrame

INFO: {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; x64; fr; rv:1.9.2.13) Gecko/20101203 Firebird/3.6.13'}


## Choose arguments

In [2]:
datetime.today().strftime('%Y-%m-%d')

'2020-04-22'

In [3]:
# advanced query
location =  None

radius = None

terms = [['brood'], ['homemade', 'zelfgemaakt']]



# limit (should always be None for our case)
limit = None

## begin and end date are very import to us
begindate = '2020-04-19'
enddate = datetime.today().strftime('%Y-%m-%d')

lang = None

query = ''
for id1, and_group in enumerate(terms):
    if id1 == 0:
        query += '('
    else:
        query += ' AND ('
    query += ' OR '.join(and_group)
    query += ')'

if location:
    query += ' near:{}'.format(location)
if radius:
    query += ' within:{}'.format(radius)
if lang:
    query += ' lang:{}'.format(lang)
if begindate:
    query += ' since:{}'.format(begindate)
if enddate:
    query += ' until:{}'.format(enddate)

print(query)


(brood) AND (homemade OR zelfgemaakt) since:2020-04-19 until:2020-04-22


In [4]:
def valid_date(s):
    try:
        return datetime.strptime(s, "%Y-%m-%d").date()
    except ValueError:
        msg = "Not a valid date: '{0}'.".format(s)
        raise argparse.ArgumentTypeError(msg)

begindate = valid_date(begindate)
enddate = valid_date(enddate)

## Search Tweets

In [5]:
tweets = query_tweets(query= query, limit=limit, begindate=begindate, enddate=enddate, poolsize=20, lang=lang)

INFO: queries: ['(brood) AND (homemade OR zelfgemaakt) since:2020-04-19 until:2020-04-22 since:2020-04-19 until:2020-04-20', '(brood) AND (homemade OR zelfgemaakt) since:2020-04-19 until:2020-04-22 since:2020-04-20 until:2020-04-21', '(brood) AND (homemade OR zelfgemaakt) since:2020-04-19 until:2020-04-22 since:2020-04-21 until:2020-04-22']
INFO: Querying (brood) AND (homemade OR zelfgemaakt) since:2020-04-19 until:2020-04-22 since:2020-04-19 until:2020-04-20
INFO: Querying (brood) AND (homemade OR zelfgemaakt) since:2020-04-19 until:2020-04-22 since:2020-04-20 until:2020-04-21
INFO: Querying (brood) AND (homemade OR zelfgemaakt) since:2020-04-19 until:2020-04-22 since:2020-04-21 until:2020-04-22
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q=(brood)%20AND%20(homemade%20OR%20zelfgemaakt)%20since%3A2020-04-19%20until%3A2020-04-22%20since%3A2020-04-19%20until%3A2020-04-20&l=None
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=def

In [6]:
print(tweets)

[<twitterscraper.tweet.Tweet object at 0x7fcb2db9ad68>, <twitterscraper.tweet.Tweet object at 0x7fcb2db9afd0>, <twitterscraper.tweet.Tweet object at 0x7fcb2db9ae10>, <twitterscraper.tweet.Tweet object at 0x7fcb2dba4160>, <twitterscraper.tweet.Tweet object at 0x7fcb2dba41d0>, <twitterscraper.tweet.Tweet object at 0x7fcb2dba4208>]


In [7]:
for tweet in tweets:
    print(tweet.text)

Feest!
En lekkers zelfgemaakt Turks brood (of een soort naan) pic.twitter.com/GYsxTWGxVF
Feest!
En lekkers zelfgemaakt Turks brood (of een soort naan) pic.twitter.com/GYsxTWGxVF
De tosti met zelfgemaakt brood is een beeeeeeetje lang pic.twitter.com/jq0wzYcIBt
Simit Turkse Sesam Bagel https://goodmoodmama.nl/simit-turkse-sesam-bagel/ … #recept #heelhollandbakt #brood #broodjesbakken #simit #breekbrood #zelfgemaakt #goedemorgen
De tosti met zelfgemaakt brood is een beeeeeeetje lang pic.twitter.com/jq0wzYcIBt
Simit Turkse Sesam Bagel https://goodmoodmama.nl/simit-turkse-sesam-bagel/ … #recept #heelhollandbakt #brood #broodjesbakken #simit #breekbrood #zelfgemaakt #goedemorgen


## Store as dataframe and drop duplicates

In [18]:
import re
def delete_hashtags(tweet):
    return re.sub('#', '', tweet)

tweets_df = DataFrame([[tweet.user_id, delete_hashtags(tweet.text), tweet.retweets, tweet.timestamp] for tweet in tweets], columns=['user_id', 'text', 'retweets', 'timestamp'])

In [19]:
tweets_df

,user_id,text,retweets,timestamp
0,6360462,Feest!\nEn lekkers zelfgemaakt Turks brood (of...,0,2020-04-19 10:46:27
1,6360462,Feest!\nEn lekkers zelfgemaakt Turks brood (of...,0,2020-04-19 10:46:27
2,989165840598818816,De tosti met zelfgemaakt brood is een beeeeeee...,0,2020-04-20 14:03:54
3,93032323,Simit Turkse Sesam Bagel https://goodmoodmama....,0,2020-04-20 06:20:43
4,989165840598818816,De tosti met zelfgemaakt brood is een beeeeeee...,0,2020-04-20 14:03:54
5,93032323,Simit Turkse Sesam Bagel https://goodmoodmama....,0,2020-04-20 06:20:43


In [20]:
tweets_df = tweets_df.drop_duplicates()

In [21]:
tweets_df

,user_id,text,retweets,timestamp
0,6360462,Feest!\nEn lekkers zelfgemaakt Turks brood (of...,0,2020-04-19 10:46:27
2,989165840598818816,De tosti met zelfgemaakt brood is een beeeeeee...,0,2020-04-20 14:03:54
3,93032323,Simit Turkse Sesam Bagel https://goodmoodmama....,0,2020-04-20 06:20:43


## Translate to english for generela sentiment analysis

In [25]:
from googletrans import Translator

In [31]:
translator = Translator()
for text in tweets_df['text']:
    translation = translator.translate(text).text
    print(translation)


Party!
And delicious homemade Turkish bread (or any kind of naan) pic.twitter.com/GYsxTWGxVF
The sandwich with homemade bread is a long beeeeeeetje pic.twitter.com/jq0wzYcIBt
Simit Turkish Sesame Bagel https://goodmoodmama.nl/simit-turkse-sesam-bagel/ recipe ... The Great British Bake Off baking bread simit break bread made good morning


## Sentiment analysis

In [32]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/arne/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [34]:
translator = Translator()
for text in tweets_df['text']:
    translation = translator.translate(text).text
    print(translation)
    print('score: {}'.format(sid.polarity_scores(translation)))

Party!
And delicious homemade Turkish bread (or any kind of naan) pic.twitter.com/GYsxTWGxVF
score: {'neg': 0.0, 'neu': 0.599, 'pos': 0.401, 'compound': 0.7712}
The sandwich with homemade bread is a long beeeeeeetje pic.twitter.com/jq0wzYcIBt
score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Simit Turkish Sesame Bagel https://goodmoodmama.nl/simit-turkse-sesam-bagel/ recipe ... The Great British Bake Off baking bread simit break bread made good morning
score: {'neg': 0.0, 'neu': 0.72, 'pos': 0.28, 'compound': 0.7906}
